In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import grader

In [262]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

download: s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt to ./stack_exchange_schema.txt


# Spark Miniproject

StackOverflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

StackExchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this dataset. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this dataset? What are our alternatives?

## Workflow

You may complete this project using the Python or Scala APIs. Most questions can be done locally, however in some cases you may want to use cloud services. See the appropriate lecture notebooks for information on how to use cloud services.

Python example:

1. Edit source code in your main.py file, classes in a separate classes.py (Class definitions need to be written in a separate file and then included at runtime.)
1. Run locally on a chunk using eg. `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`
1. Run on GCP once your testing and development are done.

Scala example:

1. Edit source code in Main.scala
1. Run the command `sbt package` from the root directory of the project
1. Use spark-submit locally on a chunk: this means adding a flag like `--master local[2]` to the spark-submit command.
1. Run on GCP once your testing and development are done.

General tips:
* SBT has some nice features, for example continuous build and test, which can greatly speed up your development.
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will also be in part-xxxxx format.
* You can access an interactive Spark/Scala REPL with `$SPARK_HOME/bin/spark-shell`.
* You can access an interactive PySpark shell with `$SPARK_HOME/bin/pyspark`.

## Accessing the data

The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three subfolders: allUsers, allPosts, and allVotes which contain chunked and gzipped xml with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

A full schema can be found [here](https://ia801500.us.archive.org/8/items/stackexchange/readme.txt).

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full dataset for every question.

You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [ ]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [ ]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing

Some rows are split across multiple lines; these can be discarded. Malformatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle xml parsing yourself using the \ selector in Scala or something like lxml.etree in Python. *Warning*: The built-in xml.etree.ElementTree behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

In [1]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
print sc.version


2.0.1


In [2]:
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [6]:
def startrow(line):
    l = line.strip().strip('\n')
    return (l.find("<row ")==0) 
def isrow(line):
    l = line.strip().strip('\n')
    return (l.find("<row ")==0 and l[-3:]==" />")
lines = sc.textFile(localpath('/spark-stats-data/allPosts/'))
totalLines = lines.count()
sus_rows = lines.filter(lambda x: startrow(x))
rows=lines.filter(lambda x: isrow(x)) 



``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python
# Command line arguments using sysv or argparse in Python
if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

## bad_xml

A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) allPosts dataset.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ascii characters.  You may need to re-encode to utf-8 (depending on your xml parser)

Note that this cleaned dataset will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [6]:
abs(rows.count()-sus_rows.count())

781

In [46]:
def bad_xml():
    return 781

grader.score(question_name='spark__bad_xml', func=bad_xml)

Your score:  1


## upvote_percentage

Each post on StackExchange can be upvoted, downvoted, and favorited. One "sanity check" we can do is to look at the ratio of upvotes to downvotes (referred to as "UpMod" and "DownMod" in the schema) as a function of how many times the post has been favorited.

You might hypothesize, for example, that posts with more favorites should have a higher upvote/downvote ratio.

Instead of looking at individual posts, we'll aggregate across number of favorites by using the post's number of favorites as our key. Since we're computing ratios, bundling together all posts with the same number of favorites effectively averages over them.  Calculate the average percentage of upvotes *(upvotes / (upvotes + downvotes))* for the first 50 ***keys***.

Do the analysis on the smaller Cross-Validated dataset.

#### Checkpoints
* Total upvotes: 313,819
* Total downvotes: 13,019
* Mean of first 50 keys (averaging the keys themselves): 24.76

In [52]:
# vote type 2 for upvotes and 3 for downvotes 

In [8]:
from lxml import etree
def favcounts(x):
    if x.find('FavoriteCount=')==-1:
        count = 0
    else:
        count=int(x[(x.find('FavoriteCount=')+15):(x.find('" Id='))])
    ID = str(x[x.find('" Id=')+6:x.find('" LastActivityDate')])
    return [ID,count]
favs = rows_post.map(favcounts)

In [7]:
def isrow(line):
    l = line.strip().strip('\n')
    return (l.find("<row ")==0 and l[-3:]==" />")
def votetype(t,line):
    return ' VoteTypeId="'+str(t)+'" ' in line
lines_post = sc.textFile(localpath('/spark-stats-data/allPosts/'))
lines_vote = sc.textFile(localpath('/spark-stats-data/allVotes/'))
rows_post=lines_post.filter(lambda x: isrow(x)) 
rows_vote=lines_vote.filter(lambda x: isrow(x)) 
upvotes = rows_vote.filter(lambda x: votetype(2,x)).map(lambda x: [str(x[(x.find('PostId=')+8):(x.find('VoteTypeId')-2)]),1])
downvotes = rows_vote.filter(lambda x: votetype(3,x)).map(lambda x: [str(x[(x.find('PostId=')+8):(x.find('VoteTypeId')-2)]),1])

In [9]:
fav_up = favs.join(upvotes).map(lambda (id, (favcount, vote)): favcount)
fav_down = favs.join(downvotes).map(lambda (id, (favcount, vote)): favcount)
up_dic = fav_up.countByValue()
down_dic = fav_down.countByValue()

In [10]:
ratio_dic = {}
for i in up_dic.keys():
    ratio_dic.update({i:(float(up_dic[i])/(up_dic[i]+down_dic[i]))})
sum(ratio_dic.keys()[0:50])/50.0

24.76

In [85]:
def upvote_percentage():
    output = []
    for i in ratio_dic.keys()[0:50]:
        output.append([i,ratio_dic[i]])
    return output

grader.score(question_name='spark__upvote_percentage', func=upvote_percentage)

Your score:  1.0


## answer_percentage

Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.

#### Checkpoints
* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [14]:
def posttype(t,line):
    return ' PostTypeId="'+str(t)+'" ' in line
def postowner(x):
    start = x.find('OwnerUserId=')+13
    if start==12:
        return '0'
    a = x.find('" ParentId=')
    if a==-1:
        end = x.find('" PostTypeId=')
        return str(x[start:end].encode('utf-8'))
    else:
        return str(x[start:a].encode('utf-8'))
questions = rows_post.filter(lambda x: posttype(1,x)).map(lambda x:postowner(x))
answers = rows_post.filter(lambda x: posttype(2,x)).map(lambda x:postowner(x))
questions.count(),answers.count()

(52060, 55304)

In [15]:
a_dic = answers.countByValue()
q_dic = questions.countByValue()

In [16]:
def user_rep(x):
    acc = str(x[x.find('" Id="')+6:x.find('" LastAccessDate=')].encode('utf-8'))
   
    if x.find('Reputation="')==-1:
        return [acc,0]
    else:
        rep = int(x[x.find('Reputation="')+12:x.find('" UpVotes=')])
        return [acc,rep]
lines_user = sc.textFile(localpath('/spark-stats-data/allUsers/'))
rows_user=lines_user.filter(lambda x: isrow(x)) 
reputation = rows_user.map(user_rep)
reputation.count()

50320

In [15]:
reputation_user = reputation.map(lambda (acc,rep):(rep,acc)).sortByKey(ascending=False)

In [16]:
topusers = reputation_user.collect()[0:99]
output = []
for user in topusers:
    ID = int(user[1])
    q = q_dic[user[1]]
    a = a_dic[user[1]]
    if a+q==0:
        rate = 0
    else:
        rate = a*1.0/(a+q)
    output.append([ID,rate])
output.append([-1,55304.0/(55304+52060)])

In [177]:
def answer_percentage():
    
    return output

grader.score(question_name='spark__answer_percentage', func=answer_percentage)

Your score:  1.0


## post_counts

If we use the total number of posts made on the site as a metric for tenure, we can look at the differences between "younger" and "older" users. You can imagine there might be many interesting features - for now just return the top 100 post counts among all users (of all types of posts) and the average reputation for every user who has that count.

In other words, aggregate the cases where multiple users have the same post count.

#### Checkpoints
* Mean of top 100 post counts: 281.51

In [17]:
posts = rows_post.map(lambda x:[postowner(x),1]).reduceByKey(lambda x,y:x+y)
total_rep =posts.join(reputation_user.map(lambda (x,y):(y,x))).map(lambda (id, (p, r)): (p,r))

In [18]:
counts = total_rep.map(lambda (x,y): x).countByValue()
reps = total_rep.reduceByKey(lambda x,y:x+y).sortByKey(ascending=False).collect()[0:100]


In [208]:
def post_counts():
    output= []
    for i in reps:
        count = i[0]
        avg = i[1]*1.0/counts[i[0]]
        output.append([count,avg])
    return output

grader.score(question_name='spark__post_counts', func=post_counts)

Your score:  1.0


## quick_answers

How long do you have to wait to get your question answered? Look at the set of ACCEPTED answers which are posted less than three hours after question creation. What is the average number of these "quick answers" as a function of the hour of day the question was asked? You should normalize by how many total accepted answers are garnered by questions posted in a given hour, just like we're counting how many quick accepted answers are garnered by questions posted in a given hour, eg. (quick accepted answers when question hour is 15 / total accepted answers when question hour is 15).

Return a list, whose ith element correspond to ith hour (e.g. 0 -> midnight, 1 -> 1:00, etc.)

*Note*: When using Scala's SimpleDateFormat class, it's important to account for your machine's local time zone. Our policy will be to use GMT: hourFormat.setTimeZone(TimeZone.getTimeZone("GMT"))

*Consider*: What biases are present in our result that we don't account for? How should we handle this?

#### Checkpoints
* Total quick accepted answers: 8,468
* Total accepted answers: 17,096

In [11]:
from datetime import datetime,timedelta


In [12]:
def acp_ans(x):
    ans_id = int(x[x.find('AcceptedAnswerId="')+18:x.find('" AnswerCount="')])
    date = x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8')
    ID = int(x[x.find('" Id="')+6:x.find('" LastActivityDate=')])
    o = datetime.strptime(date,'%Y-%m-%dT%H:%M:%S.%f')
    h = o.hour
    return [str(ID)+'|'+str(ans_id),o,h]
def ans_parent(x):
    date = x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8')
    o = datetime.strptime(date,'%Y-%m-%dT%H:%M:%S.%f')
    ID = int(x[x.find('" Id="')+6:x.find('" LastActivityDate=')])
    a = x.find('" ParentId=')
    if a==-1:
        p = 0
    else:
        p = int(x[a+12:x.find('" PostTypeId=')])
    return [str(p)+'|'+str(ID),o]
accepted = rows_post.filter(lambda x: posttype(1,x)).filter(lambda x: x.find('AcceptedAnswerId="')!=-1).map(acp_ans)
answer_parent = rows_post.filter(lambda x: posttype(2,x)).map(ans_parent)


In [21]:
rec = accepted.join(answer_parent).map(lambda (id, (o1, o2)): (o1.hour,int(abs((o1-o2)/3600)<=timedelta(0,3)))).reduceByKey(lambda x,y:x+y)
quick_hour = rec.collect()
rec_all = accepted.map(lambda (x,y,z):z).countByValue()

In [22]:
def quick_answers():
    output = [0.]*24
    for i in quick_hour:
        h = i[0]
        output[h]=i[1]*1.0/rec_all[h]
    return output

grader.score(question_name='spark__quick_answers', func=quick_answers)

Your score:  1.0


## quick_answers_full

Same as above, but on the full StackExchange dataset.

No pre-parsed data is available for this question.

#### Checkpoints
* Total quick accepted answers: 3,700,224
* Total accepted answers: 5,086,888

In [39]:
stack_post = sc.textFile(localpath('/spark-stack-data/allPosts/'))
allpost=stack_post.filter(lambda x: isrow(x)) 
accepted_stack = allpost.filter(lambda x: posttype(1,x)).filter(lambda x: x.find('AcceptedAnswerId="')!=-1).map(acp_ans)
answer_stack = allpost.filter(lambda x: posttype(2,x)).map(ans_parent)

In [34]:
rec_stack = accepted_stack.join(answer_stack).map(lambda (id, (o1, o2)): (o1.hour,int(abs((o1-o2)/3600)<=timedelta(0,3)))).reduceByKey(lambda x,y:x+y)
quick_hour_stack = rec_stack.collect()
rec_all_stack = accepted_stack.map(lambda (x,y,z):z).countByValue()

In [35]:
def quick_answers_full():
    output = [0.]*24
    for i in quick_hour_stack:
        h = i[0]
        output[h]=i[1]*1.0/rec_all_stack[h]
    return output

grader.score(question_name='spark__quick_answers_full', func=quick_answers_full)

Your score:  1.0


## identify_veterans

It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?

#### Checkpoints
* Total brief users: 24,864
* Total veteran users: 2,027

In [17]:
def post_date(x):
    owner = postowner(x)
    date = datetime.strptime(x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8'),'%Y-%m-%dT%H:%M:%S.%f')
    return [owner,date]
def acc_start(x):
    acc = str(x[x.find('" Id="')+6:x.find('" LastAccessDate=')].encode('utf-8'))
    date = datetime.strptime(x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8'),'%Y-%m-%dT%H:%M:%S.%f')
    return [acc,date]
post_all = rows_post.map(post_date)
all_users = rows_user.map(acc_start)

In [18]:

vet = post_all.join(all_users).filter(lambda (id, (o1, o2)):abs(o1-o2)>=timedelta(100,0) and abs(o1-o2)<=timedelta(150,0) ).map(lambda (id,(o1,o2)):id)

vets = vet.countByValue().keys()

In [19]:
brief = all_users.filter(lambda (x,y): x not in vets).join(post_all).map(lambda (id,(o1,o2)):id).countByValue()
briefs=brief.keys()
len(briefs)

24811

In [26]:
def q_detail(x):
    owner = postowner(x)
    date = datetime.strptime(x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8'),'%Y-%m-%dT%H:%M:%S.%f')
    if x.find('AnswerCount="')==-1:
        ac = 0
    else:
        ac = int(x[x.find('AnswerCount="')+13:x.find('" Body')])
    if x.find('Score="')==-1:
        sc = 0
    else:
        sc = int(x[x.find('Score="')+7:x.find('" Tags=')])
    if x.find('ViewCount="')==-1:
        vc = 0
    else:
        vc = int(x[x.find('ViewCount="')+11:x.find('" />')])
    if x.find('FavoriteCount=')==-1:
        fcount = 0
    else:
        fcount=int(x[(x.find('FavoriteCount=')+15):(x.find('" Id='))])
    return [owner,[date,ac,sc,vc,fcount]]
vet_firstq = rows_post.filter(lambda x: posttype(1,x)).map(detail).filter(lambda (x,y):x in vets)
bri_firstq = rows_post.filter(lambda x: posttype(1,x)).map(detail).filter(lambda (x,y):x in briefs)

In [31]:
def first_post(x,y):
    if x[0]<y[0]:
        return x
    else:
        return y
def total(x,y):
    return [x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3]]
vet_fq = vet_firstq.reduceByKey(lambda x,y:first_post(x,y)).map(lambda (x,y):('vet',y[2:]))
bri_fq = bri_firstq.reduceByKey(lambda x,y:first_post(x,y)).map(lambda (x,y):('brief',y[2:]))

In [28]:
num_vets = vet_fq.count()

In [29]:
num_briefs = bri_fq.count()

In [32]:
total_vet_data = vet_fq.reduceByKey(total).collect()[0][1]
for i in total_vet_data:
    print i*1.0/num_vets

1.29753424658
3.54082191781
923.796712329
1.29917808219


In [33]:
total_brief_data = bri_fq.reduceByKey(total).collect()[0][1]
for i in total_brief_data:
    print i*1.0/num_briefs

0.970662905501
2.10061118947
553.620592384
0.575787494123


In [84]:
def identify_veterans():
    return {"vet_score": 3.54082191781,
            "vet_views": 923.796712329,
            "vet_answers": 1.29753424658,
            "vet_favorites": 1.29917808219,
            "brief_score": 2.10061118947,
            "brief_views": 553.620592384,
            "brief_answers": 0.970662905501,
            "brief_favorites": 0.575787494123
           }

grader.score(question_name='spark__identify_veterans', func=identify_veterans)

Your score:  1.0


## identify_veterans_full

Same as above, but on the full StackExchange dataset.

No pre-parsed data is available for this question.

#### Checkpoints
* Total brief users: 1,848,628
* Total veteran users: 288,285

In [14]:
import lxml.etree as ET


In [15]:
def detail(x):
    root = ET.fromstring(x)
    if 'OwnerUserId' not in root.attrib:
        owner='0'
        cat = 'NA'
    else:
        owner = root.get('OwnerUserId')
        if owner in vets_all:
            cat = 'vet'
        elif owner in briefs_all:
            cat = 'brief'
        else:
            cat='NA'
    date = datetime.strptime(x[x.find('CreationDate="')+14:x.find('CreationDate="')+37].encode('utf-8'),'%Y-%m-%dT%H:%M:%S.%f')
    if 'AnswerCount' not in root.attrib:
        ac = 0
    else:
        ac = int(root.get('AnswerCount'))
    if 'Score' not in root.attrib:
        sc = 0
    else:
        sc = int(root.get('Score'))
    if 'ViewCount' not in root.attrib:
        vc = 0
    else:
        vc = int(root.get('ViewCount'))
    if 'FavoriteCount' not in root.attrib:
        fcount = 0
    else:
        fcount = int(root.get('FavoriteCount'))
    return [owner,[date,cat,ac,sc,vc,fcount,1]]

In [59]:
stack_post = sc.textFile(localpath('/spark-stack-data/allPosts/'))
stack_user = sc.textFile(localpath('/spark-stack-data/allUsers/'))
allpost=stack_post.filter(lambda x: isrow(x)) 
alluser=stack_user.filter(lambda x: isrow(x)) 
post_stack = allpost.map(post_date)
users_stack = alluser.map(acc_start)
vet_stack = post_stack.join(users_stack).filter(lambda (id, (o1, o2)):abs(o1-o2)>=timedelta(100,0) and abs(o1-o2)<=timedelta(150,0) ).map(lambda (id,(o1,o2)):id)
vets_all = vet_stack.countByValue().keys()
brief_stack = users_stack.filter(lambda (x,y): x not in vets_all).join(post_stack).map(lambda (id,(o1,o2)):id).countByValue()
briefs_all=brief_stack.keys()

In [91]:
import dill

dill.dump(vets_all,open('all_vets.pkd','w'))
dill.dump(briefs_all,open('all_briefs.pkd','w'))

In [17]:
import dill
vets_all = dill.load(open('all_vets.pkd','r'))
briefs_all = dill.load(open('all_briefs.pkd','r'))

In [ ]:
def first_post(x,y):
    if x[1]<y[1]:
        return x
    else:
        return y

def total_full(x,y):
    return [x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]]
firstq_stack = sc.parallelize(stack_post.filter(lambda x: isrow(x) and posttype(1,x)).map(detail).reduceByKey(first_post).map(lambda (x,y):(y[0],y[2:])).reduceByKey(total_full).collect())
stack_data=firstq_stack.collect()

In [19]:
import grader
def identify_veterans_full():
    return {"vet_score":2.156145241083955,
            "vet_views":1801.041589751163,
            "vet_answers": 1.8337194491619286,
            "vet_favorites": 0.8651969652218858,
            "brief_score": 1.1947695334768675,
            "brief_views": 1124.7777816213463,
            "brief_answers": 1.5152378277257206,
            "brief_favorites":0.375504906545935
           }
grader.score(question_name='spark__identify_veterans_full', func=identify_veterans_full)

Your score:  1.0


## word2vec

Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each StackExchange post as documents (this uses the full dataset). Use Spark ML's implementation of Word2Vec (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).

#### Parameters

The dimensionality of the vector space should be 100. The random seed should be 42L.

#### Checkpoints
* Mean of the top 25 cosine similarities: 0.7785175901170094

In [ ]:
def get_tags(x):
    if x.find('Tags="')==-1:
        return []
    else:
        tag = x[x.find('Tags="')+6:x.find('" Title=')]
        tag=tag[4:-4]
        tag = tag.split('&gt;&lt;')
        return tag

In [ ]:
from pyspark.mllib.feature import Word2Vec
stack_post = sc.textFile(localpath('/spark-stack-data/allPosts/'))
allpost=stack_post.filter(lambda x: isrow(x)) 
post_tag = allpost.map(get_tags)
word2vec = Word2Vec().setVectorSize(100).setSeed(42)
model = word2vec.fit(post_tag)
synonyms = model.findSynonyms('ggplot2', 25)
syncs = []
for word, cosine_distance in synonyms:
    syncs.append([word.encode('utf-8'),cosine_distance])

In [ ]:
def word2vec():
    return syncs

grader.score(question_name='spark__word2vec', func=word2vec)

## classification

We'd like to see if we can predict the tags of a question from its body text. Technically this is a multi-label classification problem, but to simplify things we'll use a one-vs-all approach where we choose the top k most common tags and train k binary classifiers where the labels indicate the presence or absence of that tag.

Use a logistic regression model as your classifer.

Since we can't reliably save and load models, return a list of 100 tuples ("string", [number, number, number,...]) where "string" is the tag and the numbers are your model's predicted probabilities for class 0 (eg. 0.2 means a prediction that the tag is present) across the test set.

Note that this will require some digging into the result DataFrame to extract.
The length of these probability lists is equal to the length of the test set: 4649.

#### Parameters

* [Training](s3://dataincubator-course/spark-stats-data/posts_train.zip) and [test](s3://dataincubator-course/spark-stats-data/posts_test.zip) sets are available on S3.
* Tokenize the body text into words
* number of tags to consider k = 100

In [ ]:
import dill
result= dill.load(open('class.pkd','r'))

In [ ]:
def classification():
    return result

grader.score(question_name='spark__classification', func=classification)

## k_means (ungraded)

From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*